In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from cv2 import imread, resize
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight


import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras import layers
from keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
SIZE = 224
X , y =[],[]

label_map={
    'pizza':1,
    'not_pizza':0
}

In [ ]:
for label, label_id in label_map.items():
    image_folder = f"/kaggle/input/pizza-not-pizza/pizza_not_pizza/{label}/"
    image_files = os.listdir(image_folder)
    for fname in image_files:
        img = imread(image_folder + fname)
        img_resized = resize(img, (SIZE, SIZE))
        X.append(img_resized)
        y.append(label_id)

In [ ]:
x = np.array(X) #Storing images in an array
y = np.array(y)  #Storing labels in an array

In [ ]:
# Convert y to categorical (one-hot encoding)
y = to_categorical(y, num_classes=2)

In [ ]:
#Splitting dataset in train , test  and validation
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=2)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=2)

In [ ]:
import numpy as np

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Now you can check the shape
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


**Defining the Model-**

In [ ]:
!ping -c 3 google.com


In [ ]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping , ModelCheckpoint
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten
from tensorflow.keras.layers import BatchNormalization


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16

l2_reg = regularizers.l2(0.001)  # You can adjust this value



vgg16height = 224
vgg16width = 224
batch_size = 32

basevgg16model = VGG16(input_shape=(vgg16height,vgg16width,3),
                       include_top=False,
                      weights='imagenet')

for layer in basevgg16model.layers[:-1]:
    layer.trainable = False
for layer in basevgg16model.layers[1:]:
    layer.trainable = True
    
basevgg16model.trainable = False

vgg16model = Sequential()
vgg16model.add(basevgg16model)
vgg16model.add(GlobalAveragePooling2D())

vgg16model.add(Dense(1024, activation='relu', kernel_initializer='he_normal',
                kernel_regularizer=l2_reg))
vgg16model.add(BatchNormalization())
vgg16model.add(Dropout(0.5))

vgg16model.add(Dense(512, activation='relu', kernel_initializer='he_normal',
                kernel_regularizer=l2_reg))
vgg16model.add(BatchNormalization())
vgg16model.add(Dropout(0.5))

vgg16model.add(Dense(2, activation='softmax'))  # Binary classification


In [ ]:
vgg16model.summary()

In [ ]:
# Initialize ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, shear_range=0.2)
datagen.fit(X_train)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',     # You can also use 'val_accuracy'
    patience=5,             # Stop after 5 epochs with no improvement
    restore_best_weights=True,  # Roll back to the best model
    verbose=1
)


In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_acc',
                              patience = 3,
                              verbose = 1,
                              factor = 0.5,
                              min_lr = 0.00001)

In [ ]:
vgg16model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
history = vgg16model.fit(datagen.flow(X_train,y_train), batch_size = 32,
                                 epochs = 25,
                                 validation_data=(X_val, y_val),
                                 verbose = 1,
                                 callbacks = [reduce_lr,early_stop])

In [ ]:
# Predict the labels for the test set
y_pred = vgg16model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

In [ ]:
# Calculate F1 Score
f1_score_value = f1_score(y_true_labels, y_pred_labels, average='weighted')
print(f"F1 Score (Weighted): {f1_score_value}")

In [ ]:
# Calculate Accuracy
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f"Accuracy: {accuracy}")

In [ ]:
# Evaluate the model on the test set
# test_loss, test_acc, test_precision, test_recall = best_model.evaluate(X_test, y_test)
test_loss, test_acc, test_precision, test_recall = vgg16model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')

In [ ]:
# Generate classification report
# y_pred = np.argmax(best_model.predict(X_test), axis=1)
y_pred = np.argmax(vgg16model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

In [ ]:
# Confusion Matrix
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', linewidths=0.5, 
            xticklabels=list(label_map.keys()), 
            yticklabels=list(label_map.keys()))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()